In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import tensorflow as tf
import warnings
import os
from datetime import datetime, timedelta
import joblib

warnings.filterwarnings('ignore')

class PM25Predictor:
    def __init__(self):
        self.model = None
        self.scaler = None
        self.selected_features = None
        self.df = None
        self.setup_features()
        
    def setup_features(self):
        """Setup feature names matching the training script"""
        self.selected_features = [
            # Core pollutants
            'PM2.5', 'O3', 'CO', 'NO2', 'SO2', 
            # Weather
            'Temperature', 'Humidity',
            # Time patterns
            'sin_hour', 'cos_hour', 'sin_day', 'cos_day', 'is_weekend',
            # Interactions
            'temp_humidity', 'O3_NO2_ratio', 'pollutant_index',
            # Lags (most important)
            'PM2.5_lag_1h', 'PM2.5_lag_2h', 'PM2.5_lag_3h', 'PM2.5_lag_6h', 'PM2.5_lag_12h', 'PM2.5_lag_24h',
            # Rolling stats (key ones)
            'PM2.5_mean_3h', 'PM2.5_mean_6h', 'PM2.5_mean_12h', 'PM2.5_mean_24h',
            'PM2.5_std_3h', 'PM2.5_std_12h',
            # Trends
            'PM2.5_diff_1h', 'PM2.5_diff_3h', 'temp_diff', 'humidity_diff'
        ]
        
    def load_data_and_model(self):
        """Load and prepare data, load trained model"""
        print("Dang tai du lieu va model...")
        
        # Load data
        if not os.path.exists('Air Quality Ho Chi Minh City.csv'):
            print("Khong tim thay file 'Air Quality Ho Chi Minh City.csv'")
            return False
            
        df = pd.read_csv('Air Quality Ho Chi Minh City.csv', parse_dates=['date'], dayfirst=True)
        df = df[df['Station_No'] == 2].sort_values('date')
        
        # Select and clean data (same as training script)
        cols = ['PM2.5', 'O3', 'CO', 'NO2', 'SO2', 'Temperature', 'Humidity']
        df = df[['date'] + cols].copy()
        
        for col in cols:
            if df[col].dtype == object:
                df[col] = df[col].str.replace('[^0-9.]', '', regex=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
        # Handle missing values (same as training)
        for col in cols:
            df[col] = df[col].interpolate(method='linear', limit=6)
            if df[col].isnull().sum() > 0:
                hourly_median = df.groupby(df['date'].dt.hour)[col].median()
                missing_mask = df[col].isnull()
                for idx in missing_mask[missing_mask].index:
                    hour = df.loc[idx, 'date'].hour
                    if not pd.isna(hourly_median[hour]):
                        df.loc[idx, col] = hourly_median[hour]
            df[col] = df[col].ffill().bfill()
            
        # Feature engineering (same as training)
        df['hour'] = df['date'].dt.hour
        df['day_of_week'] = df['date'].dt.dayofweek
        df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
        
        df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['sin_day'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['cos_day'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        
        df['temp_humidity'] = df['Temperature'] * df['Humidity']
        df['O3_NO2_ratio'] = df['O3'] / (df['NO2'] + 1e-6)
        df['pollutant_index'] = df['CO'] + df['NO2'] + df['SO2']
        
        # Lag features
        for lag in [1, 2, 3, 6, 12, 24]:
            df[f'PM2.5_lag_{lag}h'] = df['PM2.5'].shift(lag)
            
        # Rolling statistics
        for window in [3, 6, 12, 24]:
            df[f'PM2.5_mean_{window}h'] = df['PM2.5'].rolling(window=window, center=True).mean()
            df[f'PM2.5_std_{window}h'] = df['PM2.5'].rolling(window=window, center=True).std()
            
        # Rate of change
        df['PM2.5_diff_1h'] = df['PM2.5'].diff(1)
        df['PM2.5_diff_3h'] = df['PM2.5'].diff(3)
        df['temp_diff'] = df['Temperature'].diff(1)
        df['humidity_diff'] = df['Humidity'].diff(1)
        
        df.dropna(inplace=True)
        self.df = df
        
        # Load or create scaler
        try:
            self.scaler = joblib.load('scaler.pkl')
            print("Da tai scaler tu file")
        except:
            print("Khong tim thay scaler, tao moi...")
            self.scaler = StandardScaler()
            self.scaler.fit(df[self.selected_features])
            joblib.dump(self.scaler, 'scaler.pkl')
            
        # Load model
        try:
            self.model = load_model('optimized_pm25_model.h5', compile=False)
            print("Da tai model thanh cong")
            return True
        except:
            print("Khong tim thay model 'optimized_pm25_model.h5'")
            print("Hay chay script training truoc de tao model")
            return False
            
    def get_latest_data_sequence(self, n_steps=36):
        """Get the latest 36 hours of data for prediction"""
        if len(self.df) < n_steps:
            return None, None
            
        latest_data = self.df[self.selected_features].iloc[-n_steps:].values
        latest_timestamp = self.df['date'].iloc[-1]
        
        scaled_data = self.scaler.transform(latest_data)
        sequence = scaled_data.reshape(1, n_steps, len(self.selected_features))
        
        return sequence, latest_timestamp
        
    def predict_future_hours(self, hours_ahead=1):
        """Predict PM2.5 for specified hours ahead"""
        sequence, latest_timestamp = self.get_latest_data_sequence()
        if sequence is None:
            return None, None
            
        predictions = []
        current_sequence = sequence.copy()
        pm25_idx = self.selected_features.index('PM2.5')
        
        for i in range(hours_ahead):
            # Predict next hour
            pred_scaled = self.model.predict(current_sequence, verbose=0)[0, 0]
            
            # Convert back to actual value for storage
            dummy_array = np.zeros((1, len(self.selected_features)))
            dummy_array[0, pm25_idx] = pred_scaled
            pred_actual = self.scaler.inverse_transform(dummy_array)[0, pm25_idx]
            predictions.append(pred_actual)
            
            # Update sequence for next prediction (only if not last iteration)
            if i < hours_ahead - 1:
                # Get the last row of current sequence and create new row
                last_row = current_sequence[0, -1, :].copy()
                
                # Update PM2.5 with prediction
                last_row[pm25_idx] = pred_scaled
                
                # Update time-based features for next hour
                next_time = latest_timestamp + timedelta(hours=i+1)
                hour = next_time.hour
                day_of_week = next_time.weekday()
                
                # Calculate new time features (not scaled yet)
                sin_hour_val = np.sin(2 * np.pi * hour / 24)
                cos_hour_val = np.cos(2 * np.pi * hour / 24)
                sin_day_val = np.sin(2 * np.pi * day_of_week / 7)
                cos_day_val = np.cos(2 * np.pi * day_of_week / 7)
                is_weekend_val = float(day_of_week >= 5)
                
                # Create a dummy row with all original features to transform properly
                # We'll use the last actual data row as template
                last_actual_data = self.df.iloc[-1:][self.selected_features].copy()
                
                # Update time features in the template
                last_actual_data.iloc[0, self.selected_features.index('sin_hour')] = sin_hour_val
                last_actual_data.iloc[0, self.selected_features.index('cos_hour')] = cos_hour_val
                last_actual_data.iloc[0, self.selected_features.index('sin_day')] = sin_day_val
                last_actual_data.iloc[0, self.selected_features.index('cos_day')] = cos_day_val
                last_actual_data.iloc[0, self.selected_features.index('is_weekend')] = is_weekend_val
                
                # Transform the updated row
                new_row_scaled = self.scaler.transform(last_actual_data.values)[0]
                
                # Update PM2.5 with our prediction
                new_row_scaled[pm25_idx] = pred_scaled
                
                # Shift sequence and add new row
                current_sequence = np.roll(current_sequence, -1, axis=1)
                current_sequence[0, -1, :] = new_row_scaled
                
        timestamps = [latest_timestamp + timedelta(hours=i+1) for i in range(hours_ahead)]
        return predictions, timestamps
        
    def display_menu(self):
        """Display prediction options menu"""
        print("\n" + "="*60)
        print("DU DOAN CHAT LUONG KHONG KHI PM2.5 - HO CHI MINH CITY")
        print("="*60)
        print("Chon khoang thoi gian du doan:")
        print("1. Du doan 1 gio tiep theo")
        print("2. Du doan 6 gio tiep theo")
        print("3. Du doan 12 gio tiep theo")
        print("4. Du doan 24 gio tiep theo (1 ngay)")
        print("5. Du doan 48 gio tiep theo (2 ngay)")
        print("6. Du doan 72 gio tiep theo (3 ngay)")
        print("7. Tuy chinh so gio")
        print("8. Xem thong tin du lieu hien tai")
        print("0. Thoat")
        print("-" * 60)
        
    def get_air_quality_level(self, pm25_value):
        """Get air quality level based on PM2.5 value"""
        if pm25_value <= 12:
            return "Tot", "green"
        elif pm25_value <= 35.4:
            return "Vua phai", "yellow"
        elif pm25_value <= 55.4:
            return "Khong tot cho nhom nhay cam", "orange"
        elif pm25_value <= 150.4:
            return "Khong tot", "red"
        elif pm25_value <= 250.4:
            return "Rat khong tot", "purple"
        else:
            return "Nguy hiem", "maroon"
            
    def display_predictions(self, predictions, timestamps):
        """Display prediction results in a nice format"""
        print("\n" + "="*80)
        print("KET QUA DU DOAN PM2.5")
        print("="*80)
        
        current_pm25 = self.df['PM2.5'].iloc[-1]
        current_level, _ = self.get_air_quality_level(current_pm25)
        
        print(f"Du lieu cuoi cung: {self.df['date'].iloc[-1].strftime('%d/%m/%Y %H:%M')}")
        print(f"PM2.5 cuoi cung: {current_pm25:.1f} ug/m3 - {current_level}")
        print("-" * 80)
        print("DU DOAN CHO CAC GIO TIEP THEO:")
        print("-" * 80)
        
        for i, (pred, timestamp) in enumerate(zip(predictions, timestamps)):
            level, color = self.get_air_quality_level(pred)
            change = pred - current_pm25
            trend = "Tang" if change > 0 else "Giam" if change < 0 else "Giu nguyen"
            
            print(f"Thoi gian: {timestamp.strftime('%d/%m/%Y %H:%M')} (+{i+1}h)")
            print(f"   PM2.5 du doan: {pred:.1f} ug/m3 - {level}")
            print(f"   {trend} - Thay doi so voi cuoi: {change:+.1f} ug/m3")
            
            # Add trend compared to previous prediction for multi-hour predictions
            if i > 0:
                prev_change = pred - predictions[i-1]
                prev_trend = "Tang" if prev_change > 0 else "Giam" if prev_change < 0 else "Giu nguyen"
                print(f"   {prev_trend} - So voi gio truoc: {prev_change:+.1f} ug/m3")
            print()
            
        # Summary statistics (only show if more than 1 prediction)
        if len(predictions) > 1:
            avg_pred = np.mean(predictions)
            max_pred = np.max(predictions)
            min_pred = np.min(predictions)
            max_level, _ = self.get_air_quality_level(max_pred)
            min_level, _ = self.get_air_quality_level(min_pred)
            
            print("-" * 80)
            print("THONG KE TONG QUAN:")
            print(f"   Trung binh: {avg_pred:.1f} ug/m3")
            print(f"   Cao nhat: {max_pred:.1f} ug/m3 - {max_level}")
            print(f"   Thap nhat: {min_pred:.1f} ug/m3 - {min_level}")
            
            # Add trend analysis
            if len(predictions) >= 3:
                first_half_avg = np.mean(predictions[:len(predictions)//2])
                second_half_avg = np.mean(predictions[len(predictions)//2:])
                overall_trend = "Xu huong tang" if second_half_avg > first_half_avg else "Xu huong giam" if second_half_avg < first_half_avg else "Xu huong on dinh"
                print(f"   {overall_trend} trong khoang thoi gian du doan")
                
        print("="*80)
        
    def plot_predictions(self, predictions, timestamps):
        """Create a simple plot of predictions"""
        try:
            plt.figure(figsize=(12, 6))
            
            # Get recent actual data for context
            recent_hours = min(48, len(self.df))
            recent_data = self.df[['date', 'PM2.5']].iloc[-recent_hours:]
            
            # Plot recent actual data
            plt.plot(recent_data['date'], recent_data['PM2.5'], 
                    'b-', linewidth=2, label='Du lieu thuc te', alpha=0.8)
            
            # Plot predictions
            plt.plot(timestamps, predictions, 
                    'r-', linewidth=2, label='Du doan', marker='o', markersize=4)
            
            # Add AQI level bands
            plt.axhline(y=12, color='green', linestyle='--', alpha=0.5, label='Tot')
            plt.axhline(y=35.4, color='yellow', linestyle='--', alpha=0.5, label='Vua phai')
            plt.axhline(y=55.4, color='orange', linestyle='--', alpha=0.5, label='Khong tot cho nhom nhay cam')
            plt.axhline(y=150.4, color='red', linestyle='--', alpha=0.5, label='Khong tot')
            
            plt.title('Du doan PM2.5 - TP.HCM', fontsize=14, fontweight='bold')
            plt.xlabel('Thoi gian')
            plt.ylabel('PM2.5 (ug/m3)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            # Save plot
            plt.savefig(f'pm25_prediction_{len(predictions)}h.png', dpi=300, bbox_inches='tight')
            print(f"\nDa luu bieu do: pm25_prediction_{len(predictions)}h.png")
            plt.show()
            
        except Exception as e:
            print(f"Khong the tao bieu do: {e}")
            
    def show_current_data_info(self):
        """Show information about current data"""
        print("\n" + "="*60)
        print("THONG TIN DU LIEU HIEN TAI")
        print("="*60)
        
        latest_record = self.df.iloc[-1]
        print(f"Du lieu moi nhat: {latest_record['date'].strftime('%d/%m/%Y %H:%M')}")
        print(f"Tram do: Station 2 - TP. Ho Chi Minh")
        print()
        
        print("CHAT LUONG KHONG KHI:")
        pm25_level, _ = self.get_air_quality_level(latest_record['PM2.5'])
        print(f"   PM2.5: {latest_record['PM2.5']:.1f} ug/m3 - {pm25_level}")
        print(f"   O3: {latest_record['O3']:.1f} ug/m3")
        print(f"   CO: {latest_record['CO']:.1f} mg/m3")
        print(f"   NO2: {latest_record['NO2']:.1f} ug/m3")
        print(f"   SO2: {latest_record['SO2']:.1f} ug/m3")
        print()
        
        print("DIEU KIEN THOI TIET:")
        print(f"   Nhiet do: {latest_record['Temperature']:.1f}°C")
        print(f"   Do am: {latest_record['Humidity']:.1f}%")
        print()
        
        print("THONG KE 24H QUA:")
        last_24h = self.df.iloc[-24:] if len(self.df) >= 24 else self.df
        print(f"   PM2.5 trung binh: {last_24h['PM2.5'].mean():.1f} ug/m3")
        print(f"   PM2.5 cao nhat: {last_24h['PM2.5'].max():.1f} ug/m3")
        print(f"   PM2.5 thap nhat: {last_24h['PM2.5'].min():.1f} ug/m3")
        print("="*60)
        
    def run(self):
        """Main application loop"""
        if not self.load_data_and_model():
            return
            
        while True:
            self.display_menu()
            
            try:
                choice = input("Nhap lua chon cua ban: ").strip()
                
                if choice == '0':
                    print("\nCam on ban da su dung! Hen gap lai!")
                    break
                    
                elif choice == '8':
                    self.show_current_data_info()
                    input("\nNhan Enter de tiep tuc...")
                    continue
                    
                hours_map = {
                    '1': 1, '2': 6, '3': 12, '4': 24, '5': 48, '6': 72
                }
                
                if choice in hours_map:
                    hours = hours_map[choice]
                elif choice == '7':
                    try:
                        hours = int(input("Nhap so gio muon du doan (1-168): "))
                        if not (1 <= hours <= 168):
                            print("So gio phai tu 1 den 168!")
                            continue
                    except ValueError:
                        print("Vui long nhap so hop le!")
                        continue
                else:
                    print("Lua chon khong hop le!")
                    continue
                    
                # Perform prediction
                print(f"\nDang du doan cho {hours} gio tiep theo...")
                predictions, timestamps = self.predict_future_hours(hours)
                
                if predictions is not None:
                    self.display_predictions(predictions, timestamps)
                    
                    # Ask if user wants to see chart
                    show_chart = input("\nBan co muon xem bieu do khong? (y/n): ").lower().strip()
                    if show_chart in ['y', 'yes', 'co']:
                        self.plot_predictions(predictions, timestamps)
                        
                    # Ask if user wants to save results
                    save_results = input("\nBan co muon luu ket qua khong? (y/n): ").lower().strip()
                    if save_results in ['y', 'yes', 'co']:
                        results_df = pd.DataFrame({
                            'Timestamp': timestamps,
                            'PM2.5_Prediction': predictions,
                            'Air_Quality_Level': [self.get_air_quality_level(p)[0] for p in predictions]
                        })
                        filename = f'pm25_prediction_{hours}h_{datetime.now().strftime("%Y%m%d_%H%M")}.csv'
                        results_df.to_csv(filename, index=False)
                        print(f"Da luu ket qua: {filename}")
                        
                else:
                    print("Khong the thuc hien du doan!")
                    
            except KeyboardInterrupt:
                print("\n\nChuong trinh da bi dung. Tam biet!")
                break
            except Exception as e:
                print(f"\nCo loi xay ra: {e}")
                
            input("\nNhan Enter de tiep tuc...")

if __name__ == "__main__":
    predictor = PM25Predictor()
    predictor.run()